In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as efficientnet_preprocess
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D


In [ ]:
# Dataset paths
base_dir = r"E:\Research\dev\New_Dataset\New DataSet_object"

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.3,  # 70% train, 30% test split
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Train and test generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

test_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

def train_model(base_model, name, train_data, test_data, dropout_rate=0.3, epochs=5):
    """
    Function to train a model with a specified base and evaluate performance.
    """
    # Add custom layers on top of base model
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dropout(dropout_rate),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile with a reduced learning rate
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Add a learning rate scheduler
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    # Train the model
    history = model.fit(
        train_data,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        epochs=epochs,
        validation_data=test_data,
        validation_steps=test_data.samples // test_data.batch_size,
        callbacks=[reduce_lr]  # Include the learning rate scheduler
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(test_data, steps=test_data.samples // test_data.batch_size)
    print(f"{name} Test Accuracy: {test_acc}")

    # Save history for visualization
    return model, history


In [ ]:
# Load pre-trained ResNet50 model
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_model, resnet50_history = train_model(resnet50, "ResNet50", train_generator, test_generator)
 

In [ ]:
def smooth_curve(values, weight=0.8):
    smoothed = []
    last = values[0]
    for value in values:
        smoothed_value = last * weight + (1 - weight) * value
        smoothed.append(smoothed_value)
        last = smoothed_value
    return smoothed

def plot_history(history, model_name):
    plt.figure(figsize=(12, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(smooth_curve(history.history['accuracy']), label='Train Accuracy')
    plt.plot(smooth_curve(history.history['val_accuracy']), label='Validation Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(smooth_curve(history.history['loss']), label='Train Loss')
    plt.plot(smooth_curve(history.history['val_loss']), label='Validation Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


# Example: Plot for ResNet50
plot_history(resnet50_history, "ResNet50")

In [41]:
# Save the best model
best_model = resnet50_model  # Replace with the model of your choice
best_model.save('ResNet_copra_detection_model.h5')

# Load the model for testing
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('Resnet_copra_detection_model.h5')

# Test with a single image
test_image_path = r"E:\Research\dev\New_Dataset\New DataSet_object\test1.jpeg"
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)
if prediction[0][0] > 0.5:
    print("Prediction: Copra")
else:
    print("Prediction: Non-Copra")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Non-Copra
